# Model Fitting

## Load Dataset

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# To save dict
import pickle

In [2]:
# load dataset
h_price = pd.read_csv("King_County_House_prices_dataset.csv")#, na_values=['?'])

# Edit Dataset

In [3]:
### Change date to useful data eg date, month, year
h_price['date_m'] = np.array([i.split('/')[0] for i in h_price.date.values]).astype(int)
h_price['date_d'] = np.array([i.split('/')[1] for i in h_price.date.values]).astype(int)
h_price['date_y'] = np.array([i.split('/')[2] for i in h_price.date.values]).astype(int)

In [4]:
# 33 bedrooms, coordinates show small house, outlier, not useful for analysis
h_price.bedrooms.argmax()
h_price[15856:15857]
##### remove false bedrooms
h_price = h_price.drop(h_price.index[h_price.bedrooms.argmax()])
#h_price.sort_values('bedrooms', ascending=False).head(3)

In [5]:
#h_price.plot.scatter(x='sqft_living', y='price');
# linear dependence, one obvious outlier
h_price = h_price.drop(h_price.index[h_price.sqft_living.argmax()])

In [160]:
# Turn Nans in waterfront to 0
h_price.waterfront = h_price.waterfront.fillna(0)

## Functions / Imports

In [161]:
### All dummies
all_dummies = {}
for i in ['zipcode', 'condition', 'date_m', 'grade', 'waterfront', 'view', 'yr_renovated']:
    tmp = pd.get_dummies(h_price[i], prefix=i)
    all_dummies[i] = tmp.columns.values
# save dict
with open('dummies.pkl', 'wb') as f:
    pickle.dump(all_dummies, f, pickle.HIGHEST_PROTOCOL)

In [162]:
# Dataset
import sklearn
# function to shuffle dataset and divide into test and train set
from sklearn.model_selection import train_test_split
# Model
from sklearn.linear_model import LinearRegression
# Model with regularization
from sklearn.linear_model import Ridge
print('SkLearn version is {}'.format(sklearn.__version__))

SkLearn version is 0.23.2


In [163]:
def get_train_test_sets(features, dataFrame, res):
    ''' input list of features: list of strings
        input dataframe
        res: feature of interest'''
    X = dataFrame[features].values
    
    # reshape array to nFeatures, m
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
    #print(X.shape)

    y = dataFrame[res].values
    y = y.reshape(y.shape[0], 1)
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    
    dataset = {'x_train': x_train,
               'x_test': x_test,
               'y_train': y_train,
               'y_test': y_test}
    
    return dataset

def get_train_test_sets_wdummies(features, features_dummies, dataFrame, res):
    ''' input list of features: list of strings
        input list of dummy features: list of strings
        input dataframe
        res: feature of interest'''
    # load dict
    with open('dummies.pkl', 'rb') as f:
        all_dummies = pickle.load(f)
    ## Add dummies to features / data frame
    copy = dataFrame.copy()
    # Reset index to concat
    copy = copy.reset_index(drop=True)
    for feat_dum in features_dummies:
        df = pd.get_dummies(copy[feat_dum], prefix=feat_dum)
        # Add all missing dummy values
        dummies = all_dummies[feat_dum]
        # initializing dummies with zeros
        df_all = pd.DataFrame(0, index=np.arange(df.shape[0]), columns = dummies)
        # Add all existing ones in correct order -> loop over full one
        for feat in df.columns.values:
            df_all[feat] = df[feat]
        #df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
        features = features + list(df_all.columns.values)
        copy = pd.concat([copy, df_all], axis=1, sort=False)

    X = copy[features].values
    
    # reshape array to nFeatures, m
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
    #print(X.shape)

    y = dataFrame[res].values
    y = y.reshape(y.shape[0], 1)
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    
    dataset = {'x_train': x_train,
               'x_test' : x_test,
               'y_train': y_train,
               'y_test' : y_test}
    
    return dataset

# MAPE mean absolute percentage error
def mape(a,b):
    """ calculate MAPE, input (y_true, y_pred)"""
    mask = a != 0
    return (np.fabs(a-b)/a)[mask].mean()

def get_mape_from_dataset(dataset):
    ''' input dataset, fit model to train_set, get mape of model for train, test_set'''
    x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']
    reg = LinearRegression().fit(x_train, y_train)
    
    return mape(y_train, reg.predict(x_train)), mape(y_test, reg.predict(x_test))

def get_combinations(full_list):
    """ return possible combinations of features in list"""
    import itertools
    combinations = []
    # append all values
    full_list = list(full_list)
    if 'price' in full_list:
        full_list.remove('price')
    combinations.append(full_list)
    for i in range(1, len(full_list)):
        possib = list(itertools.combinations(full_list, r=i))
        for x in possib:
            combinations.append(list(x))
    print('There are {} combinations'.format(len(combinations)))
    
    return combinations

# Model functions:
#    reg = LinearRegression().fit(x_train, y_train)                Fits the model to (x_train, y_train)
#    reg.score(x_train, y_train), reg.score(x_test, y_test)        Get R^2 of the model
#    reg.coef_, reg.intercept_                                     Get coefficients
#    reg.predict(x_test)                                           Predict results from model


# Test:
dataset = get_train_test_sets(['sqft_living', 'grade'], h_price, 'price')
x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']
reg = LinearRegression().fit(x_train, y_train)
print(reg.score(x_train, y_train), reg.score(x_test, y_test))

0.5370480325728262 0.52769201345116


## Calculate all R^2

In [166]:
l = []
# for given features
for feature in ['bedrooms', 'price', 'bathrooms', 'sqft_living',
                'sqft_lot', 'floors', 'condition', 'grade',
                'sqft_above', 'yr_built',
                'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'date_m',
                'date_d', 'date_y', 'waterfront']:
    dataset = get_train_test_sets([feature], h_price, 'price')
    #print(feature)
    x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']
    reg = LinearRegression().fit(x_train, y_train)
    l.append([feature, reg.score(x_train, y_train)])
# sort and print all R2
l = sorted(l,key=lambda x: x[1], reverse=True)
for i in l:
    print('R^2 for feature {} is {:.2f}'.format(i[0], i[1]))

R^2 for feature price is 1.00
R^2 for feature sqft_living is 0.50
R^2 for feature grade is 0.45
R^2 for feature sqft_above is 0.36
R^2 for feature sqft_living15 is 0.35
R^2 for feature bathrooms is 0.28
R^2 for feature bedrooms is 0.10
R^2 for feature lat is 0.09
R^2 for feature waterfront is 0.07
R^2 for feature floors is 0.06
R^2 for feature sqft_lot is 0.01
R^2 for feature sqft_lot15 is 0.01
R^2 for feature zipcode is 0.00
R^2 for feature yr_built is 0.00
R^2 for feature condition is 0.00
R^2 for feature long is 0.00
R^2 for feature date_m is 0.00
R^2 for feature date_y is 0.00
R^2 for feature date_d is 0.00


* sqft_living, grade, sqft_above, sqft_living15, bathrooms = big corr
* bedrooms, floors = low corr
* zipcode, condition, date_m test with dummies (also grade because it's exponential)

## Dummy test

In [167]:
l = []
# for given features
for feature in ['zipcode', 'condition', 'date_m', 'grade', 'view', 'yr_renovated']:
    tmp = h_price.copy()
    df = pd.get_dummies(tmp[feature])
    features = df.columns.values
    tmp = tmp.join(df)
    dataset = get_train_test_sets(features, tmp, 'price')
    #print(feature)
    x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']
    reg = LinearRegression().fit(x_train, y_train)
    l.append([feature, reg.score(x_train, y_train)])
# sort and print results
l = sorted(l,key=lambda x: x[1], reverse=True)
for i in l:
    print('R^2 for feature {} is {:.2f}'.format(i[0], i[1]))

R^2 for feature grade is 0.52
R^2 for feature zipcode is 0.41
R^2 for feature view is 0.17
R^2 for feature yr_renovated is 0.03
R^2 for feature condition is 0.01
R^2 for feature date_m is 0.00


* grade correlates better as dummy, because it's not linear
* zipcode also very good correlation
* view correlation not bad
* condition, date_m no single correlation, check in combination

## Intuitive Model
* use all values from above with good R^2

In [243]:
features = ['sqft_living', 'sqft_above', 'bathrooms', 'waterfront']
dummies = ['grade', 'zipcode', 'view']

dataset = get_train_test_sets_wdummies(features, dummies, h_price, 'price')
result = get_mape_from_dataset(dataset)
print('Mape for training set {:.3f} and test set {:.3f}'.format(result[0], result[1]))

Mape for training set 0.177 and test set 0.178


* values for train / test set almost the same -> no overfit, no regularization needed
* test with less values

In [244]:
features = ['sqft_living']
dummies = ['grade', 'zipcode', 'view']

dataset = get_train_test_sets_wdummies(features, dummies, h_price, 'price')
result = get_mape_from_dataset(dataset)
print('Mape for training set {:.3f} and test set {:.3f}'.format(result[0], result[1]))

Mape for training set 0.177 and test set 0.178


* good first result, but most features do nothing relevant

### Create Model to use

In [185]:
class house_price:
    def __init__(self):
        # standard settings
        self.features = ['sqft_living']
        self.dummies = ['grade', 'zipcode', 'view']
    def fit(self, dataFrame):
        """ fit model, input dataFrame """
        dataset = get_train_test_sets_wdummies(self.features, self.dummies, dataFrame, 'price')
        x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']

        model = LinearRegression().fit(x_train, y_train)
        self.model = model
        
    def predict_list(self, x):

        # Predict if rich or poor
        y_pred = self.model.predict(x)
        
        return y_pred
    
    def predict(self, inp):
        """ input pandas file with columns features and dummies"""
        
        features = self.features
        # load dict
        with open('dummies.pkl', 'rb') as f:
            all_dummies = pickle.load(f)
        ## Add dummies to features / data frame
        copy = inp.copy()
        # Reset index to concat
        copy = copy.reset_index(drop=True)
        for feat_dum in self.dummies:
            df = pd.get_dummies(copy[feat_dum], prefix=feat_dum)
            # Add all missing dummy values
            dummies = all_dummies[feat_dum]
            # initializing dummies with zeros
            df_all = pd.DataFrame(0, index=np.arange(df.shape[0]), columns = dummies)
            # Add all existing ones in correct order -> loop over full one
            for feat in df.columns.values:
                df_all[feat] = df[feat]
            #df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
            features = features + list(df_all.columns.values)
            copy = pd.concat([copy, df_all], axis=1, sort=False)

        X = copy[features].values

        # reshape array to nFeatures, m
        if len(X.shape) == 1:
            X = X.reshape(X.shape[0], 1)
        
        return self.model.predict(X)

## Remove Outliers by STD for fit

In [180]:
# Loop over all features
price_std = h_price.copy()
# remove everything that lies x times out of std
x = 2.5
# remove from following features
for i in ['price', 'sqft_living']:
    std = price_std[i].std()
    mean = price_std[i].mean()
    print('from {} {} observations are removed'.format(i, price_std[abs(price_std[i] - mean) > std*x][i].count()))
    price_std = price_std[abs(price_std[i] - mean) < std*x]
print('{} observations are left'.format(price_std.shape))

from price 565 observations are removed
from sqft_living 405 observations are removed
(20625, 24) observations are left


In [245]:
model = house_price()

# fit model with price_std
model.fit(price_std)

dataset = get_train_test_sets_wdummies(features, dummies, price_std, 'price')

# test model with full dataset
dataset = get_train_test_sets_wdummies(features, dummies, h_price, 'price')
x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']
result = mape(y_test, model.predict_list(x_test))
print('Mape for test set {:.3f}'.format(result))
#reg.score(x_test, y_test)

# Save model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)

Mape for test set 0.161


* better fit, better result

In [250]:
# Test
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
    
test = h_price.copy()
test = test[['sqft_living', 'grade', 'zipcode', 'view', 'price']]
test = test.sample(n=10).reset_index(drop=True)
test['price_pred'] = model.predict(test)
test['APE'] = abs(test.price - test.price_pred)/test.price*100
test = test.round(0)
test

#from pandas.plotting import table
#ax = plt.subplot(111, frame_on=False) # no visible frame
#ax.xaxis.set_visible(False) # hide xaxis
#ax.yaxis.set_visible(False)
#plt.figure(figsize=(10,10))
#table(ax, test)
#plt.tight_layout()
#plt.savefig('plots/table3.png')

,sqft_living,grade,zipcode,view,price,price_pred,APE
0,3830,9,98122,1.0,2050000.0,1012225.0,51.0
1,3390,8,98116,3.0,915000.0,902764.0,1.0
2,3030,9,98119,0.0,879000.0,968408.0,10.0
3,1570,8,98199,0.0,494000.0,633144.0,28.0
4,1690,8,98024,0.0,457000.0,435502.0,5.0
5,1980,8,98199,0.0,759000.0,684105.0,10.0
6,1810,8,98022,0.0,279000.0,301160.0,8.0
7,2420,8,98059,0.0,505000.0,467454.0,7.0
8,1920,7,98023,0.0,210000.0,242101.0,15.0
9,2800,8,98028,0.0,515000.0,541388.0,5.0


In [ ]:
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
model.predict(dataFrame)

## Richmans- / Poormansmodel
Try model with two fits, one for cheap and one for expensive houses

In [109]:
class richpoor:
    def __init__(self):
        # standard settings
        self.rp_coeff = 1030000
        self.rp_range = 15000
        self.features = ['sqft_living', 'sqft_above', 'bathrooms']
        self.dummies = ['grade', 'zipcode', 'view']
    def fit_rp(self, dataFrame):
        """ fit model to decide rich or poor """
        dataset = get_train_test_sets_wdummies(self.features, self.dummies, dataFrame, 'price')
        x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']

        rp_model = LinearRegression().fit(x_train, y_train)
        self.rp_model = rp_model
        
        df_rich = dataFrame[dataFrame.price > self.rp_coeff - self.rp_range]
        if df_rich.shape[0] < 2:
            print("range or coeff is set badly")

        dataset = get_train_test_sets_wdummies(self.features, self.dummies, df_rich, 'price')
        x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']

        model_rich = LinearRegression().fit(x_train, y_train)
        self.model_rich = model_rich
        
        df_poor = dataFrame[dataFrame.price < self.rp_coeff + self.rp_range]
        if df_poor.shape[0] < 2:
            print("range or coeff is set badly")

        dataset = get_train_test_sets_wdummies(self.features, self.dummies, df_poor, 'price')
        x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']

        model_poor = LinearRegression().fit(x_train, y_train)
        self.model_poor = model_poor
        
    def predict(self, x_test):

        # Predict if rich or poor
        y_pred = self.rp_model.predict(x_test)
        mask_rich = (y_pred > self.rp_coeff)
        mask_poor = (y_pred <= self.rp_coeff)

        # Calculate all predicts
        y_poor = self.model_poor.predict(x_test)
        y_rich = self.model_rich.predict(x_test)

        # Get the correct values from both sets
        y_pred = y_rich * mask_rich + y_poor * mask_poor
        
        return y_pred

In [64]:
# Get datasets
dataset = get_train_test_sets_wdummies(features, dummies, h_price, 'price')
x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']

In [65]:
model = richpoor()
model.fit_rp(h_price)
mape(y_test, model.predict(x_test))

0.16039578384439485

In [80]:
# Test different settings
model = richpoor()
l = []
count = 0
#range1 = list(range(int(h_price.price.min())+10000, int(h_price.price.max())-1000000, 100000))
#range2 = [100, 300, 500, 1000, 3000, 5000, 10000, 100000, 500000, 1000000, 3000000]
#### best mape for coefficients (1088000, 10000) is 0.15616
range1 = list(range(900000, 2000000, 10000))
range2 = [5000, 15000, 1000]
num = len(range1) * len(range2)
for rp_coeff in range1:
    for rp_range in range2:
        count += 1
        print('{:.2f}% done!'.format(count/num*100), end='\r')
        model.rp_coeff = rp_coeff
        model.rp_range = rp_range
        model.fit_rp(h_price)
        mp = mape(y_test, model.predict(x_test))
        l.append([(rp_coeff, rp_range), mp])

In [81]:
# Get best model parameters
l = sorted(l,key=lambda x: x[1])#, reverse=True)
for i in l:
    print('mape for coefficients {} is {:.5f}'.format(i[0], i[1]))

mape for coefficients (1030000, 15000) is 0.15552
mape for coefficients (1170000, 5000) is 0.15600
mape for coefficients (1170000, 1000) is 0.15600
mape for coefficients (1180000, 5000) is 0.15612
mape for coefficients (1180000, 1000) is 0.15612
mape for coefficients (1160000, 15000) is 0.15612
mape for coefficients (940000, 1000) is 0.15613
mape for coefficients (1080000, 15000) is 0.15616
mape for coefficients (1140000, 15000) is 0.15621
mape for coefficients (1190000, 15000) is 0.15623
mape for coefficients (1100000, 5000) is 0.15630
mape for coefficients (1100000, 1000) is 0.15630
mape for coefficients (1120000, 5000) is 0.15636
mape for coefficients (1120000, 1000) is 0.15636
mape for coefficients (1020000, 5000) is 0.15640
mape for coefficients (1020000, 1000) is 0.15640
mape for coefficients (1150000, 5000) is 0.15640
mape for coefficients (1150000, 1000) is 0.15640
mape for coefficients (950000, 1000) is 0.15640
mape for coefficients (1090000, 5000) is 0.15642
mape for coeffici

In [96]:
# Loop over all features
price_std = h_price.copy()
x = 2.5
for i in ['price', 'sqft_living']:
    std = price_std[i].std()
    mean = price_std[i].mean()
    print('{} {}'.format(i, price_std[abs(price_std[i] - mean) > std*x][i].count()))
    price_std = price_std[abs(price_std[i] - mean) < std*x]
print(price_std.shape)

price 565
sqft_living 405
(20625, 24)


In [110]:
model = richpoor()
model.fit_rp(price_std)

features = ['sqft_living', 'sqft_above', 'bathrooms']
dummies = ['grade', 'zipcode', 'view']
model.features = features
model.dummies = dummies
dataset = get_train_test_sets_wdummies(features, dummies, h_price, 'price')
x_train, y_train, x_test, y_test = dataset['x_train'], dataset['y_train'], dataset['x_test'], dataset['y_test']
mape(y_test, model.predict(x_test))

0.15509822965282283

* value only slightly better, linear regression is good to fit model